In [ ]:
from utils.random_board import generate_random_board
from utils.stockfish import gest_best_move
import os

board = generate_random_board()
print(board)
print(board.legal_moves)
move = gest_best_move(board, 10)
print(move)


In [ ]:
import tensorflow as tf

tensor_test = tf.random.uniform((10000,14,8,8), minval=0, maxval=2, dtype=tf.int32)
tensor_test_2 = tf.random.uniform((10000,14,8,8), minval=0, maxval=2, dtype=tf.int32)

In [ ]:
import chess
import numpy as np

def convert_square_to_coordinates(square: chess.Square):
    # ex h1 -> (7, 0)
    print(square)
    return (7 - square // 8, square % 8)

print(convert_square_to_coordinates(chess.H1))


In [6]:
from utils.process_board import convert_board_to_array
from utils.random_board import generate_random_board
from utils.stockfish import get_score
from utils.process_tensors import save_tensor, load_tensor, concat_tensors
import numpy as np
import tensorflow as tf

done = False
def generate_data(n=100, N=25, folder_name=''):
    save_dir = f"data/{folder_name}/"
    input_path = save_dir + "input"
    output_path = save_dir + "output"

    for k in range(n):
        if done:
            print('done')
            break
        input_array = np.zeros((N, 7, 8, 8), dtype=np.int32)
        output_array = np.zeros((N,), dtype=np.float32)
        for i in range(N):
            board = generate_random_board()
            input_array[i] = convert_board_to_array(board)
            output_array[i] = get_score(board, 10)

        input_tensor = tf.convert_to_tensor(input_array)
        output_tensor = tf.convert_to_tensor(output_array)
        try:
            loaded_input = load_tensor(input_path, out_type=tf.int32)
            loaded_output = load_tensor(output_path, out_type=tf.float32)

            input_tensor = concat_tensors([input_tensor, loaded_input], axis=0)
            output_tensor = concat_tensors([output_tensor, loaded_output], axis=0)

        except Exception as e:
            # print('erreur', e)
            pass
        save_tensor(input_tensor, input_path)
        save_tensor(output_tensor, output_path)
        
        print(folder_name, (k+1)*N)

In [7]:
import threading
threading.Thread(target=generate_data, args=(4,25, 'data1')).start()
threading.Thread(target=generate_data, args=(4,25, 'data2')).start()
threading.Thread(target=generate_data, args=(4,25, 'data3')).start()
threading.Thread(target=generate_data, args=(4,25, 'data4')).start()

data3 25
data4 25
data2 25
data1 25


In [8]:
done = True

data3 50
done
data4 50
done
data2 50
done
data1 50
done


In [16]:
load_dir = "data/test/"
input_path = load_dir + "input"
output_path = load_dir + "output"

input_tensor = load_tensor(input_path, out_type=tf.int32)
output_tensor = load_tensor(output_path, out_type=tf.float32)

In [17]:
output_tensor
normalized_tensor = tf.linalg.normalize(output_tensor, axis=0, ord='euclidean', name=None)[0]
print(normalized_tensor.shape)


(19790,)


In [ ]:
# print max value of tensor

print(normalized_tensor.shape)
maxi = tf.math.reduce_max(normalized_tensor)
mini = tf.math.reduce_min(normalized_tensor)
mean = tf.math.reduce_mean(normalized_tensor)
std = tf.math.reduce_std(normalized_tensor)
 
print('max', maxi)
print('min', mini)
print('mean', mean)
print('std', std)

test = normalized_tensor/maxi

print(tf.math.reduce_max(test))
print(tf.math.reduce_min(test))


In [ ]:
max = tf.math.reduce_max(output_tensor)
min = tf.math.reduce_min(output_tensor)
std = tf.math.reduce_std(output_tensor)

abs_max = tf.math.maximum(tf.math.abs(max), tf.math.abs(min))
print(abs_max)
print(output_tensor/abs_max)
